In [ ]:
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    datasets==2.10.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2 \
    transformers \
    langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 60.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1

In [ ]:
import os
import torch
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ""

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

model_id = "intfloat/e5-base-v2"

# initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id).to(device)
model.eval()

def embed(docs: list[str]) -> list[list[float]]:
    docs = [f"passage: {d}" for d in docs]
    # tokenize
    tokens = tokenizer(
        docs, padding=True, max_length=512, truncation=True, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        # process with model for token-level embeddings
        out = model(**tokens)
        # mask padding tokens
        last_hidden = out.last_hidden_state.masked_fill(
            ~tokens["attention_mask"][..., None].bool(), 0.0
        )
        # create mean pooled embeddings
        doc_embeds = last_hidden.sum(dim=1) / \
            tokens["attention_mask"].sum(dim=1)[..., None]
    return doc_embeds.cpu().numpy()

Using cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Replace the csv file with translated comments

In [ ]:
import pandas as pd
comments_df = pd.read_csv('processed_comments.csv', engine = 'python')

In [ ]:
comments_df = comments_df[comments_df['Comment'].str.len() > 50]
comments_df['comments'] = comments_df['channelTitle'] + ' '+ comments_df['Comment']
comments_df.drop(columns = ['Comment', 'channelTitle'], inplace = True)

comments_df['index'] = comments_df.index
process_df = comments_df.copy()

,title,description,comments,index
0,im running for prime minister to give you back...,im running for prime minister to give you back...,pierre poilievre cant wait 2 more years we nee...,0
1,im running for prime minister to give you back...,im running for prime minister to give you back...,pierre poilievre pierre we will vote for you ...,1
3,im running for prime minister to give you back...,im running for prime minister to give you back...,pierre poilievre i do have freedom and i dont ...,3
4,im running for prime minister to give you back...,im running for prime minister to give you back...,pierre poilievre im a liberal but im voting co...,4
5,im running for prime minister to give you back...,im running for prime minister to give you back...,pierre poilievre scrap the bill or law that pr...,5
...,...,...,...,...
1103636,helping young people across canada,march 3 2020\nthrough the youth employment and...,justin trudeau prime minister of canada when ...,1103636
1103638,helping young people across canada,march 3 2020\nthrough the youth employment and...,justin trudeau prime minister of canada i hav...,1103638
1103644,renewing the canadian coast guard fleet,may 22 2019\nthe coast guard fleet is supporte...,justin trudeau prime minister of canada too l...,1103644
1103645,kitsilano coast guard base enhances ocean prot...,july 29 2019\nby reopening and renovating the ...,justin trudeau prime minister of canada where...,1103645


In [ ]:
from tqdm.auto import tqdm
import numpy as np

chunks = process_df["comments"]
batch_size = 512
records_processed = 0
checkpoint_interval = 50000

for i in tqdm(range(0, len(chunks), batch_size)):
    i_end = min(len(chunks), i+batch_size)
    chunk_batch = chunks[i:i_end]
    # embed current batch
    embed_batch = embed(chunk_batch)
    # add to existing np array if exists (otherwise create)
    if i == 0:
        arr = embed_batch.copy()
    else:
        arr = np.concatenate([arr, embed_batch.copy()])

    records_processed += len(chunk_batch)

    # Check if it's time to save a checkpoint
    if records_processed >= checkpoint_interval or i + batch_size >= len(chunks):
        np.save(f"embedding_checkpoint_{records_processed}.npy", arr)
        records_processed = 0

# Save the final model
np.save("embedding_model.npy", arr)


  0%|          | 0/4 [00:00<?, ?it/s]

Save the model after embedding

In [ ]:
np.save("embedding_model.npy", arr)

Load the model for querying

In [ ]:
arr = np.load("embedding_model.npy")

In [ ]:
from numpy.linalg import norm

# convert chunks list to array for easy indexing
chunk_arr = np.array(chunks)

def query(text: str, top_k: int=3) -> list[str]:
    # create query embedding
    xq = embed([f"query: {text}"])[0]
    # calculate cosine similarities
    sim = np.dot(arr, xq.T) / (norm(arr, axis=1)*norm(xq.T))
    # get indices of top_k records
    idx = np.argpartition(sim, -top_k)[-top_k:]
    docs = chunk_arr[idx]
    for d in docs.tolist():
        return d

In [ ]:
def augment_prompt(source_knowledge, query_text):
    source_knowledge = query(query_text)
    augmented_prompt = f"""Using the Youtube comments below, answer the query.
    It should be like reflecting people opinion. Do not give the comments directly, summarize the content and give the overall opinion in 2 or 3 sentences.
    Also mention the overall sentiment (Positive, Negative, Neutral) at the end like Overall Sentiment:.

    Comments:
    {source_knowledge}

    Query: {query_text}"""
    return augmented_prompt

query_text = "What are citizens praying for?"


In [ ]:
print("Prompt to Open API: ",augment_prompt(source_knowledge, query_text))

# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(source_knowledge, query_text)
)
# add to messages
messages.append(prompt)

res = chat(messages)
print("***************************************Final Result*********************************************")
print(res.content)

Prompt to Open API:  Using the Youtube comments below, answer the query.
    It should be like reflecting people opinion. Do not give the comments directly, summarize the content and give the overall opinion in 2 or 3 sentences.
    Also mention the overall sentiment (Positive, Negative, Neutral) at the end like Overall Sentiment:.

    Comments:
    pierre poilievre listen i dont care what ur gonna do but my question is are u going to kiss our heavenly fathers americans where the sun dont shine so they can help our economy

    Query: WHat are citizens praying for?
***************************************Final Result*********************************************
The comments seem to express frustration and skepticism towards relying on foreign assistance for the economy. The sentiment is Negative.
